In [1]:
from iGTP_Linear import *
from iGTP_model_modified_scvi_sample_final_update import *
from preprocess import *
from learning_utilies import *
import os 
import scanpy as sc
import pickle5 as pickle
from sklearn.manifold import TSNE
import numpy as np 

Global seed set to 0
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [2]:
model_list=os.listdir('./save_model/resilience')
model_list=[i for i in model_list if '.ipynb_checkpoints' not in i]
model_list=[i for i in model_list if 'mse' in i]
model_list=[i for i in model_list if 'brain' in i]
model_list=[i for i in model_list if 'batchbest' in i]
model_list.sort()

In [3]:
model_list

['brain_cell_type_function_AD_pathways_PC_homo_v11mse_pos_uniform_15000batchbest_fold.pt']

In [4]:
file_list=os.listdir('./save_model/resilience/processed_data')
file_list=[i for i in file_list if '.ipynb_checkpoints' not in i]
file_list=[i for i in file_list if 'brain' in i]
file_list=[i for i in file_list if 'mse' in i]
file_list=[i for i in file_list if 'batch' in i]
file_list.sort()

In [5]:
file_list

['brain_cell_type_function_AD_pathways_PC_homo_v11mse_pos_uniform_15000batch.pkl']

In [6]:
task_file_path='./save_model/resilience/processed_data/'+file_list[0]
with open(task_file_path, 'rb') as file:
    task_file = pickle.load(file)
args=task_file['args']
data_x_s=task_file['data_x_s']
mask_list_dict=task_file['mask_dict']
data=task_file['data']
TP_df=task_file['TP_df']
PPI_df=task_file['ppi_df_1']

In [7]:
model=iGTP(args,mask_list_dict)
saved_state_dict = torch.load(args['model_dir']+model_list[0],map_location="cpu")
model.load_state_dict(saved_state_dict)
model.eval()
with torch.no_grad():
    z=model.encode_z(torch.tensor(data_x_s).float())
    ppi=model.encode_ppi(torch.tensor(data_x_s).float())

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [10]:
data.obs

,orig.ident,nCount_RNA,nFeature_RNA,libraryBatch,cellBarcode,individualID,cell.type,state,percent.mt,nCount_SCT,...,cell.type.clinic.diag,cell.type.resilience.diag,new.line,new.cell.cluster,new.resilience.diag,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt
AD_200730-B41-A_ATTCGTTCAGAACTAA-1,200730-B41-A,49975.0,8937,200730-B41-A,ATTCGTTCAGAACTAA-1,R5935442,Excitatory Neurons,Exc.14,0.920521,8908.0,...,Excitatory_Neurons_Control,Excitatory_Neurons_Control,Excitatory Neurons,Excitatory Neurons,Control,8937,8937,49975.0,382.0,0.764382
AD_200730-B41-A_ATTCACTCATCTCATT-1,200730-B41-A,49856.0,9301,200730-B41-A,ATTCACTCATCTCATT-1,R2398607,Excitatory Neurons,Exc.12,1.336649,9651.0,...,Excitatory_Neurons_AD,Excitatory_Neurons_AD,Excitatory Neurons,Excitatory Neurons,AD,9301,9301,49856.0,625.0,1.253610
AD_200730-B41-A_GTTAGACAGGAACTCG-1,200730-B41-A,49567.0,9128,200730-B41-A,GTTAGACAGGAACTCG-1,R5935442,Excitatory Neurons,Exc.3,9.699439,9083.0,...,Excitatory_Neurons_Control,Excitatory_Neurons_Control,Excitatory Neurons,Excitatory Neurons,Control,9128,9128,49567.0,5470.0,11.035568
AD_200730-B41-A_CTACCTGTCTAGGCAT-1,200730-B41-A,49518.0,8024,200730-B41-A,CTACCTGTCTAGGCAT-1,R5935442,Excitatory Neurons,Exc.16,0.413731,8943.0,...,Excitatory_Neurons_Control,Excitatory_Neurons_Control,Excitatory Neurons,Excitatory Neurons,Control,8024,8024,49518.0,51.0,0.102993
AD_200730-B41-A_GATTCTTGTCATGGCC-1,200730-B41-A,49483.0,8660,200730-B41-A,GATTCTTGTCATGGCC-1,R5935442,Excitatory Neurons,Exc.4,1.377625,9001.0,...,Excitatory_Neurons_Control,Excitatory_Neurons_Control,Excitatory Neurons,Excitatory Neurons,Control,8660,8660,49483.0,640.0,1.293373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AD_200225-B10-B_CGTGTCTAGCGTCTGC-1,200225-B10-B,1066.0,833,200225-B10-B,CGTGTCTAGCGTCTGC-1,R7840275,Oligodendrocytes,Oli.11,1.989607,6735.0,...,Oligodendrocytes_Control,Oligodendrocytes_Control,Oligodendrocytes,Oligodendrocytes,Control,833,833,1066.0,11.0,1.031895
AD_200225-B10-B_TGTACAGGTCGTGGAA-1,200225-B10-B,836.0,706,200225-B10-B,TGTACAGGTCGTGGAA-1,R7840275,Oligodendrocytes,Oli.8,2.301444,6648.0,...,Oligodendrocytes_Control,Oligodendrocytes_Control,Oligodendrocytes,Oligodendrocytes,Control,706,706,836.0,11.0,1.315789
AD_200225-B10-B_ATCGTGAAGAGTGGCT-1,200225-B10-B,776.0,670,200225-B10-B,ATCGTGAAGAGTGGCT-1,R7840275,Microglia,Mic.2,2.336374,6249.0,...,Microglia_Control,Microglia_Control,Microglia,Microglia,Control,670,670,776.0,10.0,1.288660
AD_200225-B10-B_CAGGGCTCATTGTGCA-1,200225-B10-B,724.0,619,200225-B10-B,CAGGGCTCATTGTGCA-1,R7840275,Oligodendrocytes,Oli.11,3.147287,6450.0,...,Oligodendrocytes_Control,Oligodendrocytes_Control,Oligodendrocytes,Oligodendrocytes,Control,619,619,724.0,13.0,1.795580


In [11]:
TP_df_z=pd.DataFrame(z,columns=list(TP_df['TP_name']))
TP_df_z['cell_type']=data.obs['cell.type'].values
TP_df_z['diag']=data.obs['new.resilience.diag'].values
TP_df_z.to_csv('./fig_3_result/tp_df/'+file_list[0].replace('.pkl','.csv'))

In [ ]:
tsne_1 = TSNE(n_components=2, random_state=42).fit_transform(z)
label_df=data.obs.reset_index()
label_df['dim_1']=tsne_1[:,0]
label_df['dim_2']=tsne_1[:,1]

In [ ]:
label_df.to_csv('./fig_3_result/'+file_list[0].replace('.pkl','.csv'))

In [ ]:
TP_df

In [ ]:
pd.DataFrame(z,columns=list(TP_df['TP_name'])).to_csv('./fig_3_result/tp_df/'+file_list[0].replace('.pkl','.csv'))

In [ ]:
int_pathway_file='/home/vega/AD_resilience/meta_data/VEGA_TPs_of_interest.txt'

In [ ]:
int_path=pd.read_csv(int_pathway_file)
int_path=int_path.pathway.values
int_path_name=[i.upper() for i in int_path]

In [ ]:
TP_df['TP_name']=[i.upper() for i in TP_df['TP_name']]

In [ ]:
int_path_name

In [ ]:
for i in TP_df.TP_name.values:
    print(i)

In [ ]:
new_tp_df=TP_df[TP_df.TP_name.isin(int_path_name)]
print(new_tp_df)

In [ ]:
cell_inf=data.obs.copy()
cell_inf['cell_index']=range(len(cell_inf))

In [ ]:
import seaborn as sns

In [ ]:
all_emb=[]
for l in cell_inf['cell.type'].unique():
    q_index=cell_inf[(cell_inf['cell.type']==l) & (cell_inf['resilience.diag']=='Control')]['cell_index'].values
    emb_file1=z[:,new_tp_df['TP_index'].values]
    emb_file1=emb_file1[q_index,]
    emb_file1=np.mean(emb_file1,axis=0)
    all_emb.append(emb_file1)
all_emb=np.array(all_emb)
all_emb=all_emb.T
all_emb_df_1=pd.DataFrame(all_emb,columns=list(cell_inf['cell.type'].unique()))
all_emb_df_1.index=new_tp_df.TP_name.values+'_Control'
all_emb=[]
for l in cell_inf['cell.type'].unique():
        q_index=cell_inf[(cell_inf['cell.type']==l) & (cell_inf['resilience.diag']=='AD')]['cell_index'].values
        emb_file1=z[:,new_tp_df['TP_index'].values]
        emb_file1=emb_file1[q_index,]
        emb_file1=np.mean(emb_file1,axis=0)
        all_emb.append(emb_file1)
all_emb=np.array(all_emb)
all_emb=all_emb.T
all_emb_df_2=pd.DataFrame(all_emb,columns=list(cell_inf['cell.type'].unique()))
all_emb_df_2.index=new_tp_df.TP_name.values+'_AD'
all_emb_df=pd.concat([all_emb_df_1,all_emb_df_2])
    
myplot = sns.clustermap(all_emb_df.T,method='complete',metric='correlation',cmap="seismic",cbar_pos=(0.02, 0.29, 0.05, 0.08),z_score = 0,figsize=(20, 20))
myplot.ax_heatmap.set_xticklabels(myplot.ax_heatmap.get_xticklabels(), rotation=45, ha='right')
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
#sns.move_legend(myplot, "upper right")
path='./fig_3_result/cluster_heatmap/brain_ad.pdf'
plt.savefig(path, bbox_inches='tight',dpi=300)

In [ ]:
new_tp_df

In [ ]:
color_dict={'AD':'red',
            'Control': 'blue',
            }

In [ ]:
sns.set_style("white")

In [ ]:
sns.set_style("white")

In [ ]:
int_path_name_1=new_tp_df['TP_name'].values
int_path_index=new_tp_df['TP_index'].values
for m in tqdm(range(len(int_path_name_1))):
    for n in range(m+1, len(int_path_name_1)):
        name_pair = [int_path_name_1[m], int_path_name_1[n]]
        index_pair = [int_path_index[m], int_path_index[n]]
        temp_df=cell_inf[['cell.type','resilience.diag']]
        temp_df[name_pair[0]]=z[:,index_pair[0]]
        temp_df[name_pair[1]]=z[:,index_pair[1]]
        path='./fig_3_result/TP_shift_brain_df/'+name_pair[0]+'_'+name_pair[1]+'.csv'
        temp_df.to_csv(path)
        plt.figure(figsize=(50,10))
        g = sns.FacetGrid(temp_df.sample(frac=0.5), col="cell.type",hue='resilience.diag', margin_titles=True,palette=color_dict)
        g.map_dataframe(sns.scatterplot, x=name_pair[0], y=name_pair[1],s=0.8)
        figure_save_file_name='./fig_3_result/TP_shift_brain/'+name_pair[0]+'_'+name_pair[1]+'_shift.png'
        g.figure.savefig(figure_save_file_name,dpi=200)

In [ ]:
query_gene=[]
for i in new_tp_df['TP_gene']:
    for j in i:
        query_gene.append(j)
query_gene=list(set(query_gene))

In [ ]:
gene_weight=saved_state_dict['decoder_Gene_Gene.0.weight'].cpu().detach().numpy()
gene_bias=saved_state_dict['decoder_Gene_Gene.0.bias'].cpu().detach().numpy()
#ori_ppi_df=pd.read_csv('/home/iGTP/ppi/PC_homo_v11.txt',sep='\t')
#new_PPI_df=PPI_df.merge(ori_ppi_df)
new_PPI_df=PPI_df[(PPI_df.gene1.isin(query_gene)&(PPI_df.gene1!=PPI_df.gene2))]
new_PPI_df=new_PPI_df[(new_PPI_df.gene2.isin(query_gene)&(new_PPI_df.gene1!=new_PPI_df.gene2))]
new_PPI_df['PPI']=new_PPI_df['gene1']+'-'+new_PPI_df['gene2']
new_PPI_df['PPI_index']=range(len(new_PPI_df))

In [ ]:
import torch

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Set the CUDA device
    torch.cuda.set_device(1)  

In [ ]:
label_df=pd.read_csv('./fig_3_result/tSNE_df/'+file_list[0].replace('.pkl','.csv'))

In [ ]:
for i, row in tqdm(new_PPI_df.iterrows()):
    gene_1_index = row['gene_1_index']
    gene_2_index = row['gene_2_index']
    PPI=row['PPI']
    specific_sites1 = [(gene_1_index, gene_2_index)]
    specific_sites2 = [(gene_2_index, gene_1_index)]
    mask1 = np.zeros_like(gene_weight, dtype=bool)
    for site in specific_sites1:
        mask1[site] = True
    gene_weight1 = np.where(mask1, gene_weight, 0)

    mask2 = np.zeros_like(gene_weight, dtype=bool)
    for site in specific_sites2:
        mask2[site] = True
    gene_weight2 = np.where(mask2, gene_weight, 0)
    
    value1 = torch.sum(torch.mm(torch.from_numpy(ppi).cuda(),torch.from_numpy(gene_weight1.T).cuda()), dim =1).cpu().detach().numpy()
    value2 = torch.sum(torch.mm(torch.from_numpy(ppi).cuda(),torch.from_numpy(gene_weight2.T).cuda()), dim =1).cpu().detach().numpy()
    temp_df = pd.DataFrame(label_df['index'].values, columns=['cell'])
    temp_df['value1'] = value1
    temp_df['value2'] = value2
    temp_df[PPI] = np.where(temp_df['value1'] > temp_df['value2'], temp_df['value1'], temp_df['value2'])
    temp_df=temp_df[['cell',PPI]]
    temp_df.to_csv('./PPI/brain/'+PPI+'.csv',index=False)

In [ ]:
z_ppi=[]
for i in tqdm(range(len(new_PPI_df))):
    PPI=new_PPI_df.PPI.values[i]
    temp_df=pd.read_csv('./PPI/brain/'+PPI+'.csv')
    z_ppi.append(temp_df[PPI].values)
z_ppi=np.array(z_ppi).T

In [ ]:
file_list[0]

In [ ]:
label_df=pd.read_csv('./fig_3_result/tSNE_df/'+file_list[0].replace('.pkl','.csv'))

In [ ]:
tsne_1=label_df[['dim_1','dim_2']].values
cell_inf_df=label_df[['new.resilience.diag','cell.type']]

In [ ]:
res=model.compute_pair_bayes_factor(z_ppi,cell_inf_df,'new.resilience.diag','cell.type','AD','Control',tsne_1)
PPI_df_b=pd.DataFrame(res)

In [ ]:
PPI_df_b['gene_1']=new_PPI_df['gene1'].values
PPI_df_b['gene_2']=new_PPI_df['gene2'].values
PPI_df_b['Interaction']=new_PPI_df['PPI'].values
PPI_df_b['PPI_index']=range(len(PPI_df_b))

In [ ]:
PPI_df_b_1=PPI_df_b[PPI_df_b['mad']>0]
PPI_df_b_1=PPI_df_b_1[abs(PPI_df_b_1['bayes_factor'])>0.1]

In [ ]:
PPI_df_b_1_u=PPI_df_b_1.sort_values(by='mad').tail(n=20)

In [ ]:
PPI_df_b_1_u

In [ ]:
int_path_name_1=PPI_df_b_1_u['Interaction'].values
int_path_index=PPI_df_b_1_u['PPI_index'].values
for m in tqdm(range(len(int_path_name_1))):
    for n in range(m+1, len(int_path_name_1)):
        name_pair = [int_path_name_1[m], int_path_name_1[n]]
        index_pair = [int_path_index[m], int_path_index[n]]
        temp_df=cell_inf[['cell.type','resilience.diag']]
        temp_df[name_pair[0]]=z_ppi[:,index_pair[0]]
        temp_df[name_pair[1]]=z_ppi[:,index_pair[1]]
        path='./fig_3_result/PPI_shift_brain_df/'+name_pair[0]+'_'+name_pair[1]+'.csv'
        temp_df.to_csv(path)
        plt.figure(figsize=(50,10))
        g = sns.FacetGrid(temp_df.sample(frac=0.5), col="cell.type",hue='resilience.diag', margin_titles=True,palette=color_dict)
        g.map_dataframe(sns.scatterplot, x=name_pair[0], y=name_pair[1],s=0.8)
        figure_save_file_name='./fig_3_result/PPI_shift_brain/'+name_pair[0]+'_'+name_pair[1]+'_shift.png'
        g.figure.savefig(figure_save_file_name,dpi=200)

In [ ]:
label_df=pd.read_csv('./fig_3_result/tSNE_df/'+file_list[0].replace('.pkl','.csv'))

In [ ]:
ppi.shape

In [ ]:
num_processes = 10

# Parallel processing
parallel_process(new_PPI_df, process_chunk, num_processes, gene_weight, label_df)


In [ ]:
cell_inf=data.obs.copy()
cell_inf['cell_index']=range(len(cell_inf))

In [ ]:
cell_inf['cell.type'].unique()

In [ ]:
q_index=cell_inf[(cell_inf['cell.type']==l) & (cell_inf['resilience.diag']=='Control')]['cell_index'].values

In [ ]:
emb_file1=z[:,0]

In [ ]:
q_index_1=cell_inf[(cell_inf['cell.type']=='Excitatory Neurons') & (cell_inf['resilience.diag']=='Control')]['cell_index'].values
q_index_2=cell_inf[(cell_inf['cell.type']=='Excitatory Neurons') & (cell_inf['resilience.diag']=='AD')]['cell_index'].values

In [ ]:
emb_file_c=emb_file1[q_index_1,]
emb_file_a=emb_file1[q_index_2,]

In [ ]:
print(len(emb_file_c))
print(len(emb_file_a))

In [ ]:
stats.ttest_ind(emb_file_c, emb_file_a)

In [ ]:
 all_emb=[]
    for l in cell_inf['cell.type'].unique():
        q_index=cell_inf[(cell_inf['cell.type']==l) & (cell_inf['resilience.diag']=='Control')]['cell_index'].values
        emb_file1=z_tp_emb[:,new_tp_df['pathway_index'].values]
        emb_file1=emb_file1[q_index,]
        emb_file1=np.mean(emb_file1,axis=0)
        all_emb.append(emb_file1)
    all_emb=np.array(all_emb)
    all_emb=all_emb.T
    all_emb_df_1=pd.DataFrame(all_emb,columns=list(cell_inf['cell.type'].unique()))
    all_emb_df_1.index=new_tp_df.TP_name.values+'_Control'
    all_emb=[]
    for l in cell_inf['cell.type'].unique():
        q_index=cell_inf[(cell_inf['cell.type']==l) & (cell_inf['resilience.diag']=='AD')]['cell_index'].values
        emb_file1=z_tp_emb[:,new_tp_df['pathway_index'].values]
        emb_file1=emb_file1[q_index,]
        emb_file1=np.mean(emb_file1,axis=0)
        all_emb.append(emb_file1)
    all_emb=np.array(all_emb)
    all_emb=all_emb.T
    all_emb_df_2=pd.DataFrame(all_emb,columns=list(cell_inf['cell.type'].unique()))
    all_emb_df_2.index=new_tp_df.TP_name.values+'_AD'
    all_emb_df=pd.concat([all_emb_df_1,all_emb_df_2])
    
    myplot = sns.clustermap(all_emb_df,cmap="seismic",z_score = 1)
    myplot.ax_heatmap.set_xticklabels(myplot.ax_heatmap.get_xticklabels(), rotation=45, ha='right')
    path=cluster_folder+file_list[i].replace('.pkl','.png')